In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [23]:
np.random.seed(8675309)

numDays = 60
numReps = 10000
orderDelay = 4 # takes 4 days to receive an order

all_total_costs = np.zeros((numReps))
all_holding_costs = np.zeros((numReps, numDays))
all_fixed_order_costs = np.zeros((numReps, numDays))
all_unfulfilled_demand = np.zeros((numReps, numDays))

def run_sim(R=50, Q=100, initial_inventory=100):
    I = np.zeros(numDays)  # Beginning of day on-hand inventory level
    E = np.zeros(numDays) # end of day on-hand inventory level
    P = np.zeros(numDays) # pipeline inventory level

    Fixed_order_cost = np.zeros(numDays)
    Procurement_cost = np.zeros(numDays) 
    Holding_cost = np.zeros(numDays)
    Total_cost = np.zeros(numDays)
    orderQuantity = np.zeros(numDays)
    unfulfilled_demand = np.zeros(numDays)
    order_placed = np.zeros(numDays) # to track if an order was placed
    

    I[0] = initial_inventory # Initial inventory level
    P[0] = 0 # Initial pipeline inventory level

    # Generating daily random demand
    demand = np.maximum(0, np.round(np.random.normal(20, 5, numDays)))

    for day in range(numDays):
        # Check if order was placed 4 days ago
        if day >= orderDelay:
            if order_placed[day - orderDelay]:
                I[day] += orderQuantity[day - orderDelay]
                P[day] -= orderQuantity[day - orderDelay]

        if demand[day] <= I[day]:
            E[day] = I[day] - demand[day]
            unfulfilled_demand[day] = 0
            if day + 1 < numDays:
                I[day + 1] = E[day]
        else:
            unfulfilled_demand[day] = demand[day] - I[day]
            E[day] = 0
            if day + 1 < numDays:
                I[day + 1] = E[day]

        if E[day] + P[day] < R: # if eod inventory + pipeline inventory < reorder point, place an order
            # Place Order
            order_placed[day] = True
            orderQuantity[day] = Q
            Fixed_order_cost[day] = 50
            Procurement_cost[day] = orderQuantity[day] * 10
            P[day] = P[day - 1] + orderQuantity[day] # add order to pipeline inventory
            
        else:
            # No order placed
            orderQuantity[day] = 0
            Fixed_order_cost[day] = 0
            Procurement_cost[day] = 0
            P[day] = P[day - 1]

        Holding_cost[day] = .2 * E[day]
        Total_cost[day] = Fixed_order_cost[day] + Procurement_cost[day] + Holding_cost[day]

        # print("Day: ", day)
        # print('-' *20)
        # print("Beginning Inventory: ", I[day])
        # print("Demand: ", demand[day])
        # print("End Inventory: ", E[day])
        # print("Unfulfilled Demand: ", unfulfilled_demand[day])
        # print("Order Placed: ", order_placed[day])
        # print("Order Quantity: ", orderQuantity[day])
        # print("Pipeline Inventory: ", P[day])
        # print("Fixed Order Cost: ", Fixed_order_cost[day])
        # print("Procurement Cost: ", Procurement_cost[day])
        # print("Holding Cost: ", Holding_cost[day])
        # print('-' *20)
        # print("Total Cost: ", Total_cost[day])
        # print()
        # print()

    return Total_cost.sum(), Holding_cost.sum(), Fixed_order_cost.sum(), unfulfilled_demand.sum()


for rep in range(numReps):
    total_cost, holding_cost, fixed_cost, unfulfilled_demand = run_sim(R=50, Q=100, initial_inventory=100)
    all_total_costs[rep] = total_cost
    all_holding_costs[rep] = holding_cost
    all_fixed_order_costs[rep] = fixed_cost
    all_unfulfilled_demand[rep] = unfulfilled_demand

print("Average Total Cost: ", all_total_costs.mean())
print("Average Holding Cost: ", all_holding_costs.mean())
print("Average Fixed Order Cost: ", all_fixed_order_costs.mean())
print("Average Unfulfilled Demand: ", all_unfulfilled_demand.mean())





Average Total Cost:  17248.71492
Average Holding Cost:  2158.1149200000013
Average Fixed Order Cost:  718.6
Average Unfulfilled Demand:  59.7845


## Part B Answer:

Increasing the order point would:
- Decrease the holding costs
- Increase unfulfilled demand
- Decrease total inventory costs


